## https://anilcanatik.com/amazon_movies_sentiment.html

In [5]:
import csv
import json
input_file = "C:/University_Courses/NLP/Sentiment_Analysis/Movies_Review_Sentiment_Analysis_in_NLP/AmazonDataset/Movies_and_TV_5.json"
input_json = open(input_file, "r", encoding="utf-8")

output_file = "C:/University_Courses/NLP/Sentiment_Analysis/Movies_Review_Sentiment_Analysis_in_NLP/AmazonDataset/Movies_and_TV_5.csv"
with open(output_file, "w", encoding="utf-8") as output_csv:
    csv_writer = csv.writer(output_csv)
    flag = 0
    for line in input_json.readlines():
        dic = json.loads(line)
        # writing headline in the beginning
        if flag == 0:
            csv_writer.writerow(dic)
            flag = 1
        csv_writer.writerow(dic.values())
print("Done")

Done


In [2]:
import pandas as pd
import string

input_data = pd.read_csv("C:/University_Courses/NLP/Sentiment_Analysis/Movies_Review_Sentiment_Analysis_in_NLP/AmazonDataset/Movies_and_TV_5.csv")
input_data['overall'] = input_data['overall'].astype(object) # fix datatype error
input_data['reviewText'] = input_data['reviewText'].astype(object) # fix datatype error
              

In [3]:
input_data.head(3)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,ADZPIG9QOCDG5,0005019281,"Alice L. Larson ""alice-loves-books""","[0, 0]",This is a charming version of the classic Dick...,4.0,good version of a classic,1203984000,"02 26, 2008"
1,A35947ZP82G7JH,0005019281,Amarah Strack,"[0, 0]",It was good but not as emotionally moving as t...,3.0,Good but not as moving,1388361600,"12 30, 2013"
2,A3UORV8A9D5L2E,0005019281,Amazon Customer,"[0, 0]","Don't get me wrong, Winkler is a wonderful cha...",3.0,Winkler's Performance was ok at best!,1388361600,"12 30, 2013"


In [4]:
input_data.shape

(795200, 9)

In [4]:
dataset = {"reviewText": input_data["reviewText"], "overall": input_data["overall"]  }
dataset = pd.DataFrame(data = dataset)
dataset = dataset.dropna()

In [5]:
dataset.head(3)

,reviewText,overall
0,This is a charming version of the classic Dick...,4.0
1,It was good but not as emotionally moving as t...,3.0
2,"Don't get me wrong, Winkler is a wonderful cha...",3.0


In [6]:
dataset.shape

(1697472, 2)

Introducing Positive and Negative Labels:
Eliminating the neutral reviews of "3",
positive label value is +1 and it includes 4 and 5 overall ratings;
while negative label value is -1 and in includes 1 and 2 overal ratings.

In [7]:
dataset = dataset[dataset["overall"] != "3.0"] # need datatype=object
dataset["label"] = dataset["overall"].apply(lambda rating : +1 if str(rating) > '3' else -1)

In [8]:
dataset.head(3)

,reviewText,overall,label
0,This is a charming version of the classic Dick...,4.0,1
3,Henry Winkler is very good in this twist on th...,5.0,1
4,This is one of the best Scrooge movies out. H...,4.0,1


In [9]:
dataset.shape

(1496953, 3)

In [10]:
dataset.count()

reviewText    1496953
overall       1496953
label         1496953
dtype: int64

In [11]:
print("Number of positive reviews are {}, while number of negative reviews are {} in the dataset"
      .format((dataset.label == 1).sum(),(dataset.label == -1).sum()))

Number of positive reviews are 1291214, while number of negative reviews are 205739 in the dataset


dataset_i = imbalanced
dataset_b = balanced


In [12]:
dataset_i = dataset.sample(frac = 0.03, replace = False, random_state=42)

In [13]:
dataset_i.count()

reviewText    44909
overall       44909
label         44909
dtype: int64

In [14]:
print("Number of positive reviews are {}, while number of negative reviews are {} in the dataset.".format((dataset_i.label == 1).sum(),(dataset_i.label == -1).sum()))

Number of positive reviews are 38819, while number of negative reviews are 6090 in the dataset.


In [15]:
dataset_neg = dataset[dataset["label"] == -1]
dataset_pos= dataset[dataset["label"] == +1]
dataset_neg = dataset_neg.sample(frac = 0.1215, replace = False, random_state = 42)
dataset_pos = dataset_pos.sample(frac= 0.01936, replace = False, random_state = 42)
print("dataset_neg: {}, dataset_pos: {}.".format(dataset_neg.count(),dataset_pos.count()))
dataset_b = pd.concat([dataset_neg,dataset_pos])
print("dataset_b: {}".format(dataset_b.count()))

dataset_neg: reviewText    24997
overall       24997
label         24997
dtype: int64, dataset_pos: reviewText    24998
overall       24998
label         24998
dtype: int64.
dataset_b: reviewText    49995
overall       49995
label         49995
dtype: int64


In [16]:
dataset_b.head()


,reviewText,overall,label
715454,I was fortunate enough to watch this with John...,2.0,-1
201768,"Good movie, bad format. If you people would st...",1.0,-1
62207,"Awful, horribly stupid movie. One of THE most ...",1.0,-1
1658239,"I SHOULD NOT HAVE BOUGHT IT, I DID SO BECAUSE...",1.0,-1
570367,"I have always been a fan of Charlie's Angels, ...",2.0,-1


Data Cleaning Process

In [17]:
from nltk.corpus import wordnet

def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)

# clean text data_i
dataset_i["review_clean"] = dataset_i["reviewText"].apply(lambda x: clean_text(x))
# clean text data_b
dataset_b["review_clean"] = dataset_b["reviewText"].apply(lambda x: clean_text(x))


12 m 44S

In [18]:
import pickle

In [19]:
pickle.dump(dataset_i , open("dataset_i.pkl" , "wb"))
pickle.dump(dataset_b , open("dataset_b.pkl" , "wb"))

In [20]:
dataset_i = pickle.load(open('dataset_i.pkl','rb'))
dataset_b = pickle.load(open('dataset_b.pkl','rb'))

In [21]:
dataset_i.head()

,reviewText,overall,label,review_clean
181173,"The movie ""Scream 2"" is basically just like ""S...",1.0,-1,movie scream basically like scream neither mov...
1459925,"Good Movie, I liked it, Great FlimI like the d...",4.0,1,good movie like great flimi like depictation c...
56386,Oliver Stone gives Cruise his best role as Ron...,5.0,1,oliver stone give cruise best role ron kovic b...
1236023,It was difficult making this tv series into a ...,4.0,1,difficult make tv series movie series primaril...
272125,In 1946 John Wayne (1907-79) was a big star as...,2.0,-1,john wayne big star result film stagecoach dar...


In [22]:
dataset_b.head()

,reviewText,overall,label,review_clean
715454,I was fortunate enough to watch this with John...,2.0,-1,fortunate enough watch john scott shepherd aut...
201768,"Good movie, bad format. If you people would st...",1.0,-1,good movie bad format people would stop pay ju...
62207,"Awful, horribly stupid movie. One of THE most ...",1.0,-1,awful horribly stupid movie one overrate movie...
1658239,"I SHOULD NOT HAVE BOUGHT IT, I DID SO BECAUSE...",1.0,-1,buy mel gibson suppose stop watch min never see
570367,"I have always been a fan of Charlie's Angels, ...",2.0,-1,always fan charlie's angel absolutely adore dr...


WordCloud for dataset_b

Most Used Words in Negative Reviews:

In [24]:
from matplotlib import pyplot as plt
neg_reviews = dataset_b[dataset_b.label == -1]
neg_string = []
for t in neg_reviews.review_clean:
    neg_string.append(t)
neg_string = pd.Series(neg_string).str.cat(sep=' ')

from wordcloud import WordCloud

wordcloud = WordCloud(width=1600, height=800,max_font_size=200).generate(neg_string)
plt.figure(figsize=(12,10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()


ModuleNotFoundError: No module named 'wordcloud'

Most Used Words in Positive Reviews:

In [ ]:
from matplotlib import pyplot as plt
pos_reviews = dataset_b[dataset_b.label == +1]
pos_string = []
for t in pos_reviews.review_clean:
    pos_string.append(t)
pos_string = pd.Series(pos_string).str.cat(sep=' ')
from wordcloud import WordCloud

wordcloud = WordCloud(width=1600, height=800,max_font_size=200).generate(neg_string)
plt.figure(figsize=(12,10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

Splitting Dataset:


In [ ]:
from sklearn.model_selection import train_test_split
Xb = pd.DataFrame(dataset_b, columns = ["reviewText"])
yb = pd.DataFrame(dataset_b, columns = ["label"])
train_Xb, test_Xb, train_yb, test_yb = train_test_split(Xb, yb,test_size=0.30)

In [ ]:
from sklearn.model_selection import train_test_split

Xi = pd.DataFrame(dataset_i, columns = ["reviewText"])
yi = pd.DataFrame(dataset_i, columns = ["label"])

train_Xi, test_Xi, train_yi, test_yi = train_test_split(Xi, yi,test_size=0.30)

In [ ]:
train_Xb.head(3)

In [ ]:
print("Shape of the data data : train_Xb={} train_yb={} test_Xb={} test_yb={}".format(str(train_Xb.shape),str(train_yb.shape),str(test_Xb.shape),str(test_yb.shape)))

In [ ]:
print("Shape of the data data : train_Xi={} train_yi={} test_Xi={} test_yi={}".format(str(train_Xi.shape),str(train_yi.shape),str(test_Xi.shape),str(test_yi.shape)))

Vectorization:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
train_vectorb = vectorizer.fit_transform(train_Xb["reviewText"])
test_vectorb = vectorizer.transform(test_Xb["reviewText"])
print("Vocabulary size: {}".format(len(vectorizer.vocabulary_)))
print("X_train:\n{}".format(repr(train_Xb)))
print("X_test: \n{}".format(repr(test_Xb)))
feature_names = vectorizer.get_feature_names()
print("Number of features: {}".format(len(feature_names)))


In [ ]:
train_vectori = vectorizer.fit_transform(train_Xi["reviewText"])
test_vectori = vectorizer.transform(test_Xi["reviewText"])
print("Vocabulary size: {}".format(len(vectorizer.vocabulary_)))
print("X_train:\n{}".format(repr(train_Xi)))
print("X_test: \n{}".format(repr(test_Xi)))
feature_names = vectorizer.get_feature_names()
print("Number of features: {}".format(len(feature_names)))

Training Model:

In [ ]:
from sklearn.linear_model import LogisticRegression

clr = LogisticRegression()
clr.fit(train_vectorb, train_yb.values.ravel())
scores = clr.score(test_vectorb, test_yb) # accuracy
pred_yb = clr.predict(test_vectorb)
print(scores)


In [ ]:
clr.fit(train_vectori, train_yi.values.ravel())
scores = clr.score(test_vectori, test_yi) # accuracy
pred_yi = clr.predict(test_vectori)
print(scores)

Model Assesment:

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

print("Accuracy: {:.2f}%".format(accuracy_score(test_yb, pred_yb) * 100))
print("\nF1 Score: {:.2f}".format(f1_score(test_yb, pred_yb) * 100))
print("\nCOnfusion Matrix:\n", confusion_matrix(test_yb, pred_yb))

Assesment of Imbalanced Learning Model:

In [ ]:
print("Accuracy: {:.2f}%".format(accuracy_score(test_yi, pred_yi) * 100))
print("\nF1 Score: {:.2f}".format(f1_score(test_yi, pred_yi) * 100))
print("\nCOnfusion Matrix:\n", confusion_matrix(test_yi, pred_yi))

In [ ]:
def test_model(sentence):
    sen = vectorizer.transform([sentence]).toarray()
    res = clr.predict(sen)[0]
    if res == 1:
        return '+'#'Positive review'
    else:
        return '-'#'Negative review'

In [ ]:
sen = '''
I watch a lot of movies and I like to give them all a chance just in case there is something interesting or exciting to warrant a viewing Unfortunately this movie has none of these features it is pointless and offers nothing in the way of story line,acting or direction The plot is non-existent with the actors just going through the motions and the dialogue is sooo boring its embarrassing. I wish the previous reviewers had posted earlier as this would have saved me 95 mins of my time
 '''

In [ ]:
res = test_model(sen)
res_LogisticRegression = test_model_LogisticRegression(sen)
print('res =====================>',res)